Create a model using built-in library from Pytorch.
This code closelt follows nn_tutorial notebook.

In [1]:
import csv, torch, math

### read in target and attr input files
train_target_file = 'train_target_in.csv'
train_attr_file = 'train_attr_in.csv'
valid_target_file = 'valid_target_in.csv'
valid_attr_file = 'valid_attr_in.csv'

def read_data(target_file, attr_file):
    
    with open(target_file, 'r') as f: 
        reader = csv.reader(f)
        target_input_list = list(reader)

    with open(attr_file, 'r') as f: 
        reader = csv.reader(f)
        attr_input_list = list(reader)
        
    target_input = [''.join(sublist) for sublist in target_input_list]
    attr_input = [[float(x) for x in t] for t in attr_input_list]
    
    coded_y = []
    for i in target_input:
        if i == 's': 
            coded_y.append(0)
        elif i == 'x':
            coded_y.append(1)
        else:
            coded_y.append(2)


    ## convert arrays and lists into tensors for training
    return map(torch.tensor, (attr_input, coded_y))

x_train, y_train = read_data(train_target_file, train_attr_file)
x_valid, y_valid = read_data(valid_target_file, valid_attr_file)

x_train and y_train should be in the right shapes.

In [7]:
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

torch.Size([1568, 3827]) torch.Size([1568])
torch.Size([784, 3827]) torch.Size([784])


target_input_list is in list of lists. We want it to be a list of characters.
attr_input is also a list of string lists. We want it to be a list of float lists.

In [19]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim = 1)
    #print('---', preds, yb)
    return (preds == yb).float().mean()


Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [33]:
train_n, c = x_train.shape
valid_n, c = x_valid.shape
D_out = 3
lr = 0.5
epochs = 10
train_bs = 5
validate_bs = train_bs*2

import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

[Code below does not work anymore. Ignore it.] Try a simple linear combination from model() and check if the shapes are as expected.

In [12]:
preds = model(x_train) 
print(x_train.shape, preds.shape, y_train.shape)

NameError: name 'weights' is not defined

Try running loss function on the inputs

In [ ]:
loss_func(preds, y_train)

In [13]:
from torch import nn
from torch import optim

class SoundRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        
        ## This one line does the same thing as the two lines below 
        ## and [xb @ self.weights + self.bias]
        self.lin = nn.Linear(c, D_out)
#         self.weights = nn.Parameter(torch.randn(c, D_out) / math.sqrt(c))
#         self.bias = nn.Parameter(torch.zeros(D_out))
        
    def forward(self, xb):
        return self.lin(xb)

## Get the model and optim object that will be used to update model parameters
def get_model():
    model = SoundRecognition()
    return model, optim.SGD(model.parameters(), lr = lr)

In [34]:
model, opt = get_model()

def fit():
    for epoch in range(epochs):
        
        print('Training')
        model.train()
        for i in range((train_n - 1) // train_bs + 1):
            
            start_i = i * train_bs
            end_i = start_i + train_bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
        
            #print('Acc: ', accuracy(pred, yb))
        
        ####validate at each epoch
        print('Validating')
        model.eval()
        loss = []
        with torch.no_grad():
            for i in range((valid_n - 1) // validate_bs + 1):
                start_i = i * validate_bs
                end_i = start_i + validate_bs
                xb = x_valid[start_i:end_i]
                yb = y_valid[start_i:end_i]
                pred = model(xb)
                #print('----', yb)
                loss.append(loss_func(pred, yb))
        
        #print(loss)
        valid_loss = sum(loss)
        
    
        print('Epoch: ', epoch, ' | Loss: ', valid_loss)
    

fit()


Training
Validating
Epoch:  0  | Loss:  tensor(29637142.)
Training
Validating
Epoch:  1  | Loss:  tensor(23212510.)
Training
Validating
Epoch:  2  | Loss:  tensor(33642460.)
Training
Validating
Epoch:  3  | Loss:  tensor(28960350.)
Training
Validating
Epoch:  4  | Loss:  tensor(33693024.)
Training
Validating
Epoch:  5  | Loss:  tensor(30181214.)
Training
Validating
Epoch:  6  | Loss:  tensor(31183892.)
Training
Validating
Epoch:  7  | Loss:  tensor(58646732.)
Training
Validating
Epoch:  8  | Loss:  tensor(33577488.)
Training
Validating
Epoch:  9  | Loss:  tensor(30286262.)
